In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[101,  98,  29],
       [239,  98, 242],
       [  2, 156, 236]])

In [32]:
class ChargeEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Peice array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start SOC
        self.state = 20 + random.randint(-3,3)
        # Set max SOC
        self.max_state = 100
        # Set power
        self.power = 20
        # Set price data
        self.priceArray = np.float32(np.array([36.22, 27.98, 4.6, 9.38, -0.43, 2.26, 4.02, 8.48, 18.73,
                                               20.64, 22.99, 18.93, 16.67, 9.51, 3.59, 2.31, 2.78, 6.3, 14.18,
                                               16.11, 21.42, 25.87, 30.71, 18.44]))
        # Set current index
        self.currentIndex = 0
        self.currentprice = 0
        # Set temp 
        self.temp = 0
        self.change = 0
        # Set median
        self.mid = np.median(self.priceArray)
        self.open = 0
        # Set total time 
        self.time = 23 
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 sell
        # 1 -1 = 0 hold
        # 2 -1 = 1 buy 
        self.state += (action - 1) * self.power  
        self.currentprice = self.priceArray[self.currentIndex]
        # Reduce time by 1 h
        self.time -= 1 
        
        
        if 100 < self.state < (self.max_state + self.power):
            self.state = 100
        if (0 - self.power) < self.state < 0:
            self.state = 0
            
        # Calculate reward    
        # BUy
       # Give plus reward when charge at low proce, and minus at high price, and 0 at hold
        if self.currentprice < (self.mid - 2):
            temp = abs(self.currentprice) * (action - 1)
            reward = temp
            
        # Sell
        # Give minus reward when discharge at low proce, and plus at high price, and 0 at hold
        elif self.currentprice > (self.mid + 2):
            temp = -abs(self.currentprice) * (action - 1)
            reward = temp
            
        # Hold
        # Give plus reward when hold in median price range, and 0 reward otherwise
        else:
            temp = abs(abs(action) - 1) * self.mid
            reward = temp
        
        # Move to the next price
        self.currentIndex += 1
        
        # Set Low boundary and Up boundary
        #Set end state
        if self.time <= 0 and self.state == 100: 
            done = True
            reward = 20
        elif self.time <= 0 and self.state >= 90:
            done = True
            reward = 10
        elif self.time <= 0 and self.state >= 80:
            done = True
            reward = 5
        elif self.time <= 0 and self.state < 80:
            done = True
            reward = -200
        elif self.state < 0 or self.state > 100:
            done = True
            reward = -1000
        else:
            done = False       

        

        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([20 + random.randint(-3,3)])#.astype(float)
        # Set max SOC
        self.max_state = 100
        # Set power
        self.power = 20
        # Set price data
        self.priceArray = np.float32(np.array([36.22, 27.98, 4.6, 9.38, -0.43, 2.26, 4.02, 8.48, 18.73,
                                               20.64, 22.99, 18.93, 16.67, 9.51, 3.59, 2.31, 2.78, 6.3, 14.18,
                                               16.11, 21.42, 25.87, 30.71, 18.44]))
        # Set current index
        self.currentIndex = 0
        self.currentprice = 0
        # Set temp 
        self.temp = 0
        self.change = 0
        # Set median
        self.mid = np.median(self.priceArray)
        self.open = 0
        # Set total time 
        self.time = 23 
        return self.state

In [33]:
env=ChargeEnv()

In [34]:
env.observation_space.sample()

array([45.533485], dtype=float32)

In [35]:
env.reset()

array([22])

In [36]:
from stable_baselines3.common.env_checker import check_env

In [37]:
check_env(env, warn=True)

AssertionError: The observation returned by the `reset()` method does not match the given observation space

In [38]:
episodes = 50
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} State:{}'.format(episode, score, n_state))
env.close()

Episode:1 Score:-197.5599964261055 State:60
Episode:2 Score:-158.75999861955643 State:[43]
Episode:3 Score:-975.0999975204468 State:-20
Episode:4 Score:11.535001754760742 State:100
Episode:5 Score:-996.3599982261658 State:-20
Episode:6 Score:-249.1299975514412 State:[40]
Episode:7 Score:-74.85000103712082 State:80
Episode:8 Score:-963.7799987792969 State:-20
Episode:9 Score:-995.344998896122 State:-20
Episode:10 Score:-996.9699985384941 State:-20
Episode:11 Score:-963.7799987792969 State:[-20]
Episode:12 Score:-1069.4599992632866 State:-20
Episode:13 Score:-950.2000003457069 State:-20
Episode:14 Score:-1160.7199998497963 State:120
Episode:15 Score:-963.7799987792969 State:-20
Episode:16 Score:-251.83000010252 State:20
Episode:17 Score:-934.5049998760223 State:-20
Episode:18 Score:-1000.6500000357628 State:[-20]
Episode:19 Score:-963.7799987792969 State:-20
Episode:20 Score:-967.0900003314018 State:-20
Episode:21 Score:-972.0200004577637 State:[-20]
Episode:22 Score:-14.544997870922089 

In [39]:
log_path = os.path.join('Training', 'Logs')

In [40]:
model = PPO("MlpPolicy", env, learning_rate=0.0001, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [41]:
model.learn(total_timesteps=500000)

Logging to Training\Logs\PPO_23
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 14.1     |
|    ep_rew_mean     | -722     |
| time/              |          |
|    fps             | 560      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 14.2         |
|    ep_rew_mean          | -674         |
| time/                   |              |
|    fps                  | 401          |
|    iterations           | 2            |
|    time_elapsed         | 10           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0061987545 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.1         |
|    explained_variance   

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 21.4         |
|    ep_rew_mean          | -250         |
| time/                   |              |
|    fps                  | 322          |
|    iterations           | 11           |
|    time_elapsed         | 69           |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 0.0030182553 |
|    clip_fraction        | 0.0646       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.573       |
|    explained_variance   | -0.000998    |
|    learning_rate        | 0.0001       |
|    loss                 | 8.15e+03     |
|    n_updates            | 100          |
|    policy_gradient_loss | -0.00661     |
|    value_loss           | 4.14e+04     |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 22.4         |
|    ep_rew_mean          | -191         |
| time/                   |              |
|    fps                  | 309          |
|    iterations           | 21           |
|    time_elapsed         | 138          |
|    total_timesteps      | 43008        |
| train/                  |              |
|    approx_kl            | 0.0022286917 |
|    clip_fraction        | 0.0322       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.417       |
|    explained_variance   | -6.79e-05    |
|    learning_rate        | 0.0001       |
|    loss                 | 2.64e+03     |
|    n_updates            | 200          |
|    policy_gradient_loss | -0.00367     |
|    value_loss           | 1.17e+04     |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 22.9         |
|    ep_rew_mean          | -158         |
| time/                   |              |
|    fps                  | 302          |
|    iterations           | 31           |
|    time_elapsed         | 210          |
|    total_timesteps      | 63488        |
| train/                  |              |
|    approx_kl            | 0.0028036113 |
|    clip_fraction        | 0.0218       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.381       |
|    explained_variance   | -1.07e-05    |
|    learning_rate        | 0.0001       |
|    loss                 | 1.97e+04     |
|    n_updates            | 300          |
|    policy_gradient_loss | -0.00529     |
|    value_loss           | 3.5e+04      |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 22.9        |
|    ep_rew_mean          | -152        |
| time/                   |             |
|    fps                  | 304         |
|    iterations           | 41          |
|    time_elapsed         | 275         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.001138903 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.404      |
|    explained_variance   | -2.11e-05   |
|    learning_rate        | 0.0001      |
|    loss                 | 1.5e+04     |
|    n_updates            | 400         |
|    policy_gradient_loss | -0.000445   |
|    value_loss           | 2.4e+04     |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 22.8

...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 22.8         |
|    ep_rew_mean          | -57.5        |
| time/                   |              |
|    fps                  | 302          |
|    iterations           | 201          |
|    time_elapsed         | 1361         |
|    total_timesteps      | 411648       |
| train/                  |              |
|    approx_kl            | 0.0016106786 |
|    clip_fraction        | 0.0255       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.364       |
|    explained_variance   | 0            |
|    learning_rate        | 0.0001       |
|    loss                 | 5.6e+03      |
|    n_updates            | 2000         |
|    policy_gradient_loss | 0.00213      |
|    value_loss           | 1.02e+04     |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 22.7         |
|    ep_rew_mean          | -84.2        |
| time/                   |              |
|    fps                  | 302          |
|    iterations           | 211          |
|    time_elapsed         | 1430         |
|    total_timesteps      | 432128       |
| train/                  |              |
|    approx_kl            | 0.0044344435 |
|    clip_fraction        | 0.0754       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.313       |
|    explained_variance   | 0            |
|    learning_rate        | 0.0001       |
|    loss                 | 2.12e+04     |
|    n_updates            | 2100         |
|    policy_gradient_loss | -0.00608     |
|    value_loss           | 3.05e+04     |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 22.6         |
|    ep_rew_mean          | -115         |
| time/                   |              |
|    fps                  | 301          |
|    iterations           | 221          |
|    time_elapsed         | 1501         |
|    total_timesteps      | 452608       |
| train/                  |              |
|    approx_kl            | 0.0013772978 |
|    clip_fraction        | 0.0384       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.363       |
|    explained_variance   | 0            |
|    learning_rate        | 0.0001       |
|    loss                 | 1.58e+03     |
|    n_updates            | 2200         |
|    policy_gradient_loss | 0.00144      |
|    value_loss           | 1.28e+04     |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 22.9        |
|    ep_rew_mean          | -76.2       |
| time/                   |             |
|    fps                  | 298         |
|    iterations           | 231         |
|    time_elapsed         | 1583        |
|    total_timesteps      | 473088      |
| train/                  |             |
|    approx_kl            | 0.002026688 |
|    clip_fraction        | 0.0354      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.36       |
|    explained_variance   | 5.96e-08    |
|    learning_rate        | 0.0001      |
|    loss                 | 5.02e+03    |
|    n_updates            | 2300        |
|    policy_gradient_loss | 0.00216     |
|    value_loss           | 1.29e+04    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 22.9  

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 22.8         |
|    ep_rew_mean          | -80          |
| time/                   |              |
|    fps                  | 297          |
|    iterations           | 241          |
|    time_elapsed         | 1661         |
|    total_timesteps      | 493568       |
| train/                  |              |
|    approx_kl            | 0.0071519753 |
|    clip_fraction        | 0.0567       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.354       |
|    explained_variance   | 1.19e-07     |
|    learning_rate        | 0.0001       |
|    loss                 | 1.36e+04     |
|    n_updates            | 2400         |
|    policy_gradient_loss | -0.00703     |
|    value_loss           | 3.99e+04     |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

In [13]:
#evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [14]:
#model.save('PPO')

In [42]:
episodes = 100
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action, _states = model.predict(obs)
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{} State:{}'.format(episode, score, n_state))
env.close()

Episode:1 Score:-49.61000108718872 State:[82]
Episode:2 Score:-49.820000648498535 State:[82]
Episode:3 Score:10.369999885559082 State:100
Episode:4 Score:-8.189999103546143 State:100
Episode:5 Score:-15.640000104904175 State:[100]
Episode:6 Score:-214.33999919891357 State:[60]
Episode:7 Score:-206.2300000190735 State:[79]
Episode:8 Score:29.359999418258667 State:[98]
Episode:9 Score:-262.35999870300293 State:[79]
Episode:10 Score:49.555000483989716 State:100
Episode:11 Score:-28.760001182556152 State:100
Episode:12 Score:13.660000085830688 State:100
Episode:13 Score:-9.789999723434448 State:[98]
Episode:14 Score:-194.10000014305115 State:[57]
Episode:15 Score:-200.0 State:[20]
Episode:16 Score:-6.219999551773071 State:[81]
Episode:17 Score:-227.660001039505 State:[78]
Episode:18 Score:-23.199999809265137 State:[98]
Episode:19 Score:21.305000066757202 State:100
Episode:20 Score:3.720000207424164 State:[100]
Episode:21 Score:-20.339999496936798 State:[99]
Episode:22 Score:-1059.170000851

In [ ]:
env.close()

In [ ]:
#Save the model
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [ ]:
model.save(PPO_path)

In [ ]:
del model

In [ ]:
model = PPO.load('PPO_model', env=env)

In [ ]:
#View logs in Tensorboard
training_log_path = os.path.join(log_path, 'PPO')
!tensorboard --logdir={training_log_path}